## Пример использования текущей методики расчета температуры

расчет производится уже проинтегрированным способом, выражение для расчета температуры движущегося флюида предполагает неизменность следующих свойств:
$$ U, C_p, \eta, g_G, \theta, v, dv/dt, dp/dL
$$
на рассматриваемом расчетном участке

Импорт необходимых модулей

In [1]:
import sys
sys.path.append('../')

import uPVT.PVT as PVT
import Hasan_Kabir_class as HK_temp
import Tool_work_with_data as tool

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools

init_notebook_mode(connected=True)

Создание экземляров классов для корреляции и работы с данными

In [3]:
temp_cor = HK_temp.Hasan_Kabir_cor()
temp_data = tool.Data()

fluid = PVT.FluidStanding()

Основной расчет, без шагов

In [4]:

depth_m = 2550
pbh_pa=25*10**6
step_m = 10

pressure_grad_pam=pbh_pa/depth_m

temp_data.clear_data()

distance_list = list(range(1,depth_m,step_m))


for i in distance_list:
    p_pa = pbh_pa - pressure_grad_pam * i
    
    temp_geo_c = temp_cor.tei_c - temp_cor.gg_cm * i
    
    t_fluid_c = temp_cor.calc_t_c_fluid(i, p_pa)
    
    
    temp_data.get_data(temp_cor)
    
    temp_data.distance_list.append(i)
    
    temp_data.t_list.append(temp_geo_c)
    
    temp_data.p_list.append(p_pa / 10 ** 6)


Дополнительные функции для удобства построения графиков

In [5]:
def trace(number_param):
    tracep = go.Scattergl(
        x = temp_data.get_values(number_param),
        y = temp_data.distance_list,
        name = temp_data.get_name(number_param),
        mode = 'markers'
    )
    return tracep

def plot():
    #layout = dict(title = ' H(P,T)',  yaxis=dict(autorange='reversed'))
    layout = dict(title = 'Распредление температуры',  yaxis=dict(title='Расстояние от забоя, м'))
    fig = dict(data=data, layout=layout)

    iplot(fig, filename='basic-scatter')

Все данные, сохраненные с помощью инструмента и доступные для построения графиков.

In [6]:
temp_data.print_all_names()

Номер 0 для получения параметра ql_m3sec
Номер 1 для получения параметра qg_m3sec
Номер 2 для получения параметра kl_wmc
Номер 3 для получения параметра kg_wmc
Номер 4 для получения параметра rhol_kgm3
Номер 5 для получения параметра rhog_kgm3
Номер 6 для получения параметра cpl_jkgc
Номер 7 для получения параметра cpg_jkgc
Номер 8 для получения параметра mul_pas
Номер 9 для получения параметра mug_pas
Номер 10 для получения параметра rti_m
Номер 11 для получения параметра rto_m
Номер 12 для получения параметра rci_m
Номер 13 для получения параметра mu_an_pas
Номер 14 для получения параметра cp_an_jkgc
Номер 15 для получения параметра rho_an_kgm3
Номер 16 для получения параметра k_an_wmc
Номер 17 для получения параметра betta_1c
Номер 18 для получения параметра delta_temp_an_c
Номер 19 для получения параметра rco_m
Номер 20 для получения параметра rwb_m
Номер 21 для получения параметра kcem_wmc
Номер 22 для получения параметра kt_wmc
Номер 23 для получения параметра ke_wmc
Номер 24 для

Построение графиков неправильным способом, без разбивок на участки

In [7]:
trace1 = go.Scattergl(
    x = temp_data.p_list,
    y = temp_data.distance_list,
    name = 'Pressure_MPa',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = temp_data.t_list,
    y = temp_data.distance_list,
    name = 'T_geo_C',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = temp_data.get_values(52),
    y = temp_data.distance_list,
    name = temp_data.get_name(52),
    mode = 'markers'
)

data = [trace1, trace2, trace3]

plot()

### В данном расчете применяется та же формула, но расчет ведется отдельным элементам, с пересчетом некоторых PVT свойств

In [8]:
temp_cor_with_step = HK_temp.Hasan_Kabir_cor()
temp_data_with_step = tool.Data()
temp_data_with_step.clear_data()

depth_m = 2550
pbh_pa=25*10**6

pressure_grad_pam=pbh_pa/depth_m
step_m = 10
distance_list = list(range(0, depth_m, step_m))



temp_cor_with_step.tfi_c = 93.3
temp_data_with_step.last_distance_m = 0
for i in distance_list:
    p_pa = pbh_pa - pressure_grad_pam * i

    temp_geo_c = temp_cor.tei_c - temp_cor_with_step.gg_cm * i

    fluid.calc(p_pa / 10 ** 5, temp_cor_with_step.tfi_c)

    temp_cor_with_step.rhol_kgm3 = fluid.rho_oil_kgm3
    temp_cor_with_step.rhog_kgm3 = (3.5 * 10 ** (-3) * fluid.gamma_gas * p_pa /
                                    fluid.z / (fluid.t_c + 273.15))

    temp_cor_with_step.mul_pas = fluid.mu_oil_cP / 10 ** 3
    temp_cor_with_step.gas_pas = fluid.mu_gas_cP / 10 ** 3
    distance_in_this_iter = i - temp_data_with_step.last_distance_m

    t_fluid_c = temp_cor_with_step.calc_t_c_fluid(distance_in_this_iter, p_pa)

    temp_data_with_step.last_distance_m = i

    temp_data_with_step.get_data(temp_cor_with_step)

    temp_data_with_step.distance_list.append(i)

    temp_data_with_step.t_list.append(temp_geo_c)

    temp_data_with_step.p_list.append(p_pa / 10 ** 6)

    temp_cor_with_step.tfi_c = t_fluid_c
    
    temp_cor_with_step.tei_c = temp_geo_c



Различные графики и свойства.
Т.к. не все свойства пересчитываются на каждом шаге идем сближение геотермы и температуры флюида ближе к устью

In [9]:
trace1 = go.Scattergl(
    x = temp_data_with_step.p_list,
    y = temp_data_with_step.distance_list,
    name = 'P_MPa',
    mode = 'markers'
)

trace2 = go.Scattergl(
    x = temp_data_with_step.t_list,
    y = temp_data_with_step.distance_list,
    name = 'T_geo_C',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = temp_data_with_step.get_values(52),
    y = temp_data_with_step.distance_list,
    name = temp_data_with_step.get_name(52) + ' по участкам',
    mode = 'markers'
)

trace4 = go.Scattergl(
    x = temp_data.get_values(52),
    y = temp_data.distance_list,
    name = temp_data.get_name(52) + ' без участков' ,
    mode = 'markers'
)

data = [trace1, trace2, trace3, trace4]

plot()

In [10]:


trace2 = go.Scattergl(
    x = temp_data_with_step.get_values(50),
    y = temp_data_with_step.distance_list,
    name = temp_data_with_step.get_name(50) + ' по участкам',
    mode = 'markers'
)

trace3 = go.Scattergl(
    x = temp_data.get_values(50),
    y = temp_data.distance_list,
    name = temp_data_with_step.get_name(50) + ' без участков',
    mode = 'markers'
)


data = [ trace2, trace3]

plot()
